In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import os

In [2]:
def preprocess_eva_file(file_path):
    with open(file_path, 'r') as file:
        raw_data = file.readlines()

    processed_data = []
    current_game_id = None
    home_team = None
    away_team = None

    for line in raw_data:
        if line.startswith('id'):
            current_game_id = line.strip().split(',')[1]
        elif line.startswith('info') and 'visteam' in line:
            away_team = line.strip().split(',')[2]
        elif line.startswith('info') and 'hometeam' in line:
            home_team = line.strip().split(',')[2]
        elif line.startswith('play'):
            play_data = line.strip().split(',')
            play_data.insert(1, current_game_id)
            play_data.insert(2, home_team)
            play_data.insert(3, away_team)
            processed_data.append(play_data)

    return processed_data


In [3]:
def read_eva_to_dataframe(file_path):
    processed_data = preprocess_eva_file(file_path)

    df = pd.DataFrame(processed_data, columns=['event', 'game_id', 'home_team', 'away_team', 
                                               'inning', 'home_away', 'player_id', 'count', 
                                               'pitches', 'event_description'])

    df['batting_team'] = df.apply(lambda row: row['home_team'] if row['home_away'] == '1' else row['away_team'], axis=1)

    df = df.drop(columns=['event', 'inning', 'count', 'pitches'])

    return df

In [4]:
file_path = 'data/2021eve/2021ANA.EVA'
df = read_eva_to_dataframe(file_path)
df.head(50)

,game_id,home_team,away_team,home_away,player_id,event_description,batting_team
0,ANA202104010,ANA,CHA,0,andet001,K,CHA
1,ANA202104010,ANA,CHA,0,eatoa002,43/G4,CHA
2,ANA202104010,ANA,CHA,0,abrej003,S9/G34+,CHA
3,ANA202104010,ANA,CHA,0,moncy001,K,CHA
4,ANA202104010,ANA,CHA,1,fletd002,K,ANA
5,ANA202104010,ANA,CHA,1,ohtas001,K,ANA
6,ANA202104010,ANA,CHA,1,troum001,K,ANA
7,ANA202104010,ANA,CHA,0,grany001,K,CHA
8,ANA202104010,ANA,CHA,0,garcl004,K,CHA
9,ANA202104010,ANA,CHA,0,robel002,D7/L7+,CHA


In [5]:
def classify_outcome(play: str) -> str:
    if play.startswith("SB"):
        return "stolen_base"
    elif play.startswith("PO"):
        return "pickoff"
    elif play.startswith("DI"):
        return "defensive_indifference"
    elif play.startswith("FC"):
        return "fielders_choice"
    elif play.startswith("E"):
        return "error"
    elif play.startswith("WP"):
        return "wild_pitch"
    elif play.startswith("HP"):
        return "hit_by_pitch"
    elif play.startswith("IW"):
        return "intentional_walk"
    elif play.startswith("W"):
        return "walk"
    elif play.startswith("K"):
        return "strikeout"
    elif play.startswith("HR"):
        return "home_run"
    elif play.startswith("H"):
        return "home_run"
    elif play.startswith("T"):
        return "triple"
    elif play.startswith("D"):
        return "double"
    elif play.startswith("S"):
        return "single"
    else:
        return "out in play"

# Apply the classification function to the 'event' column to create a new 'outcome' column
df['outcome'] = df['event_description'].apply(classify_outcome)


In [6]:
def read_ros_to_dataframe(file_path):
    # Define the column headers for the roster DataFrame
    headers = ['player_id', 'last_name', 'first_name',
               'batting_hand', 'throwing_hand', 'batting_team', 'position']

    # Read the ROS file into a pandas DataFrame
    df = pd.read_csv(file_path, header=None, names=headers)

    # Combine the first and last names into a single column
    df['name'] = df['first_name'] + ' ' + df['last_name']

    # Drop the unnecessary columns
    df = df.drop(columns=['batting_hand', 'throwing_hand', 'first_name', 'last_name'])

    return df

In [7]:


data = []
teams = ['ANA', 'ARI', 'ATL', 'BAL', 'BOS', 'CHN', 'CHA', 'CIN', 'CLE', 'COL', 
         'DET', 'HOU', 'KCA', 'LAN', 'MIA', 'MIL', 'MIN', 'NYN', 'NYA', 'OAK', 'PHI', 
         'PIT', 'SDN', 'SFN', 'SEA', 'SLN', 'TBA', 'TEX', 'TOR', 'WAS']

# Create a single roster DataFrame for all teams
all_rosters = []
for team in teams:
    try:
        file_path = f'data/2021eve/{team}2021.ROS'
        roster = read_ros_to_dataframe(file_path)
        all_rosters.append(roster)
    except FileNotFoundError:
        print(f"Roster file not found for team {team}")

all_team_rosters = pd.concat(all_rosters, ignore_index=True)

for team in tqdm.tqdm(teams):
    try:
        # Read the EVA file into a DataFrame
        file_path = f'data/2021eve/2021{team}.EVA'
        team_data = read_eva_to_dataframe(file_path)
    except FileNotFoundError:
        # Read the EVN file into a DataFrame
        file_path = f'data/2021eve/2021{team}.EVN'
        team_data = read_eva_to_dataframe(file_path)

    # Apply the classification function to the 'event' column to create a new 'outcome' column
    team_data['outcome'] = team_data['event_description'].apply(
        classify_outcome)

    # Merge the all_team_rosters DataFrame with the team_data DataFrame using a left join
    team_data = pd.merge(team_data, all_team_rosters,
                         on=['player_id', 'batting_team'], how='left')

    # Concatenate each game's DataFrame to the data list
    data.append(team_data)

# Combine all the DataFrames in the data list into a single DataFrame
df = pd.concat(data, ignore_index=True)


100%|██████████| 30/30 [00:03<00:00,  9.54it/s]


In [8]:

# Drop duplicate records
df = df.drop_duplicates()

In [9]:
len(df)

194561

In [10]:
# count the nan values in each column
df.isna().sum()

game_id              0
home_team            0
away_team            0
home_away            0
player_id            0
event_description    0
batting_team         0
outcome              0
position             0
name                 0
dtype: int64

In [15]:
# print where kyle schwarber was out in play
df[(df['name'] == 'Kyle Schwarber') & (df['outcome'] == 'out in play')]

,game_id,home_team,away_team,home_away,player_id,event_description,batting_team,outcome,position,name
8787,ARI202105150,ARI,WAS,0,schwk001,6/F6MD,WAS,out in play,OF,Kyle Schwarber
8806,ARI202105150,ARI,WAS,0,schwk001,NP,WAS,out in play,OF,Kyle Schwarber
8856,ARI202105150,ARI,WAS,0,schwk001,4/L34D+,WAS,out in play,OF,Kyle Schwarber
8867,ARI202105160,ARI,WAS,0,schwk001,53/G4,WAS,out in play,OF,Kyle Schwarber
8886,ARI202105160,ARI,WAS,0,schwk001,8/F8D+,WAS,out in play,OF,Kyle Schwarber
...,...,...,...,...,...,...,...,...,...,...
216769,WAS202110010,WAS,BOS,0,schwk001,3/P3SF/FL,BOS,out in play,OF,Kyle Schwarber
216852,WAS202110010,WAS,BOS,0,schwk001,6/P5S,BOS,out in play,OF,Kyle Schwarber
216886,WAS202110020,WAS,BOS,0,schwk001,8/F8D,BOS,out in play,OF,Kyle Schwarber
217010,WAS202110030,WAS,BOS,0,schwk001,3/G3S,BOS,out in play,OF,Kyle Schwarber


In [12]:
# print tim anderson outcome value counts
df[df['name'] == 'Kyle Schwarber']['outcome'].value_counts()

out in play               224
strikeout                  85
walk                       57
single                     55
home_run                   32
double                     19
wild_pitch                  6
hit_by_pitch                6
error                       6
defensive_indifference      2
pickoff                     1
intentional_walk            1
stolen_base                 1
Name: outcome, dtype: int64

In [13]:
# count any duplicate rows
df.duplicated().sum()

0

In [14]:
df.head()

,game_id,home_team,away_team,home_away,player_id,event_description,batting_team,outcome,position,name
0,ANA202104010,ANA,CHA,0,andet001,K,CHA,strikeout,SS,Tim Anderson
1,ANA202104010,ANA,CHA,0,eatoa002,43/G4,CHA,out in play,OF,Adam Eaton
2,ANA202104010,ANA,CHA,0,abrej003,S9/G34+,CHA,single,1B,Jose Abreu
3,ANA202104010,ANA,CHA,0,moncy001,K,CHA,strikeout,3B,Yoan Moncada
4,ANA202104010,ANA,CHA,1,fletd002,K,ANA,strikeout,SS,David Fletcher
